<a href="https://colab.research.google.com/github/Mirsha-RG/Biblioteca/blob/main/formulas_airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install tableone > null

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
import re
import matplotlib.pyplot as plt
from tableone import TableOne
from geopy.distance import geodesic

In [ ]:
def binary(data):
  bin_reg = bin_reg = r"^[01](?:\.0)?\.?$"
  if str(data) == 'nan':
    return np.nan
  else:
    return bool(re.findall(bin_reg, str(data)))

def is_binary(df_, col):
  """
  to consider this as a pure binary var take into account that
  the others not reach a limit of range..
  #no puedehaber un porcentaje de varriable superior con más de un  digito!!!!!!
  """
  df = df_.copy()
  percent =  df[col].apply(binary).sum() / df[col].count()
  if percent > 0.5:
    return True
  else:
    return False

In [ ]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/data-preprocessing-project-tutorial/main/AB_NYC_2019.csv"

In [ ]:
df = pd.read_csv(url)

In [ ]:
def breakdown(df_):
  df = df_.copy()
  normal  = []
  nonormal = []
  binaries = []
  cates = []
  for col in df.columns:
    if df[col].dtypes.name=='int64' or df[col].dtypes.name == 'float64':
      if is_binary(df, col):
        binaries.append(col)
      else:
        if shapiro(df[col]).pvalue > 0.05:
          normal.append(col)
        else:
          nonormal.append(col)
    else:
      cates.append(col)
  return normal, nonormal, binaries +  cates

In [ ]:
normal, nonormal, cats = breakdown(df)

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 48895.
  res = hypotest_fun_out(*samples, **kwds)


In [ ]:
cats

['calculated_host_listings_count',
 'name',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'room_type',
 'last_review']

In [ ]:
to_drop = []
for var in cats:
  #print("--"*20)
  #print(var)
  #print(df[var].unique())
  #print(len(df[var].unique()))
  if len(df[var].unique()) > 7:
    to_drop.append(var)
  #print("--"*20)

In [ ]:
df.drop(columns = to_drop + ["id",	"host_id"],
        inplace=True)

In [ ]:
central_park = (40.782694, -73.966249)
df['d_central'] = df.apply(lambda col: geodesic(central_park,(col['latitude'], col['longitude'])).meters, axis =1)
df.drop(columns = ['latitude', 'longitude'], inplace=True)

In [ ]:
normal, nonormal, cats = breakdown(df)

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 48895.
  res = hypotest_fun_out(*samples, **kwds)


In [ ]:
df_tab = df.copy()
TableOne(df, nonnormal=nonormal, categorical=cats)

Missing                  Overall
n                                                                            48895
neighbourhood_group, n (%)        Bronx                                 1091 (2.2)
                                  Brooklyn                            20104 (41.1)
                                  Manhattan                           21661 (44.3)
                                  Queens                               5666 (11.6)
                                  Staten Island                          373 (0.8)
room_type, n (%)                  Entire home/apt                     25409 (52.0)
                                  Private room                        22326 (45.7)
                                  Shared room                           1160 (2.4)
price, median [Q1,Q3]                                   0       106.0 [69.0,175.0]
minimum_nights, median [Q1,Q3]                          0            3.0 [1.0,5.0]
number_of_reviews, median [Q1,Q3]                       0           5.0 [1.0,24.0]
reviews_per_month, median [Q1,Q3]                   10052            0.7 [0.2,2.0]
availability_365, median [Q1,Q3]                        0         45.0 [0.0,227.0]
d_central, median [Q1,Q3]                               0  7601.5 [4432.8,11067.0]

In [ ]:
df_tab = df[['price', 'room_type', 'neighbourhood_group', ]].copy()

In [ ]:
TableOne(df_tab, categorical = ['room_type', 'neighbourhood_group'],
         pval=True,
         groupby='room_type',
         htest_name=True)

Grouped by room_type                                                                                   
                                                        Missing        Overall Entire home/apt   Private room   Shared room P-Value           Test
n                                                                        48895           25409          22326          1160                       
price, mean (SD)                                              0  152.7 (240.2)   211.8 (284.0)   89.8 (160.2)  70.1 (101.7)  <0.001  One-way ANOVA
room_type, n (%)           Entire home/apt                        25409 (52.0)   25409 (100.0)        0 (0.0)       0 (0.0)  <0.001    Chi-squared
                           Private room                           22326 (45.7)         0 (0.0)  22326 (100.0)       0 (0.0)                       
                           Shared room                              1160 (2.4)         0 (0.0)        0 (0.0)  1160 (100.0)                       
neighbourhood_group, n (%) Bronx                                    1091 (2.2)       379 (1.5)      652 (2.9)      60 (5.2)  <0.001    Chi-squared
                           Brooklyn                               20104 (41.1)     9559 (37.6)   10132 (45.4)    413 (35.6)                       
                           Manhattan                              21661 (44.3)    13199 (51.9)    7982 (35.8)    480 (41.4)                       
                           Queens                                  5666 (11.6)      2096 (8.2)    3372 (15.1)    198 (17.1)                       
                           Staten Island                             373 (0.8)       176 (0.7)      188 (0.8)       9 (0.8)

# Results

de acuerdo a los resultados de la tab anterior o existen diferencias signf ( p <0.05) , en el precio por el typo de habitacuón ofrecida, siendo la mas costosa el piso completo sdeguido de private y comp, cin 120 dolares y 130 dolares de diferenca respectivamente.


se observa tambien que el tipo de vivienda no es independiente de  el vecindario. no obstante la mayor parte de tipois están la categorias privadas y completas.



In [ ]:
TableOne(df_tab, categorical = ['room_type', 'neighbourhood_group'],
         pval=True,
         groupby='neighbourhood_group', htest_name=True)

Grouped by neighbourhood_group                                                                                                                                           
                                                                  Missing        Overall         Bronx       Brooklyn      Manhattan        Queens  Staten Island P-Value                                       Test
n                                                                                  48895          1091          20104          21661          5666            373                                                   
price, mean (SD)                                                        0  152.7 (240.2)  87.5 (106.7)  124.4 (186.9)  196.9 (291.4)  99.5 (167.1)  114.8 (277.6)  <0.001                              One-way ANOVA
room_type, n (%)           Entire home/apt                                  25409 (52.0)    379 (34.7)    9559 (47.5)   13199 (60.9)   2096 (37.0)     176 (47.2)  <0.001                                Chi-squared
                           Private room                                     22326 (45.7)    652 (59.8)   10132 (50.4)    7982 (36.8)   3372 (59.5)     188 (50.4)                                                   
                           Shared room                                        1160 (2.4)      60 (5.5)      413 (2.1)      480 (2.2)     198 (3.5)        9 (2.4)                                                   
neighbourhood_group, n (%) Bronx                                              1091 (2.2)  1091 (100.0)        0 (0.0)        0 (0.0)       0 (0.0)        0 (0.0)  <0.001  Chi-squared (warning: expected count < 5)
                           Brooklyn                                         20104 (41.1)       0 (0.0)  20104 (100.0)        0 (0.0)       0 (0.0)        0 (0.0)                                                   
                           Manhattan                                        21661 (44.3)       0 (0.0)        0 (0.0)  21661 (100.0)       0 (0.0)        0 (0.0)                                                   
                           Queens                                            5666 (11.6)       0 (0.0)        0 (0.0)        0 (0.0)  5666 (100.0)        0 (0.0)                                                   
                           Staten Island                                       373 (0.8)       0 (0.0)        0 (0.0)        0 (0.0)       0 (0.0)    373 (100.0)

In [ ]:
df[normal + nonormal].corr(method='spearman')

,price,minimum_nights,number_of_reviews,reviews_per_month,availability_365,d_central
price,1.000000,0.101289,-0.055009,-0.018527,0.086088,-0.308825
minimum_nights,0.101289,1.000000,-0.174905,-0.288797,0.075981,-0.094886
number_of_reviews,-0.055009,-0.174905,1.000000,0.706208,0.236664,0.083548
reviews_per_month,-0.018527,-0.288797,0.706208,1.000000,0.392126,0.088209
availability_365,0.086088,0.075981,0.236664,0.392126,1.000000,0.060698
d_central,-0.308825,-0.094886,0.083548,0.088209,0.060698,1.000000
